# Simple QA Agentic Flow with crewAI

Source:

Examples for crewAI
https://github.com/crewAIInc/crewAI-examples/tree/main

Notebooks:
https://github.com/crewAIInc/crewAI-examples/tree/main/Notebooks

Notebook (modified):
https://github.com/crewAIInc/crewAI-examples/blob/main/Notebooks/Flows/Simple%20QA%20Crew%20%2B%20Flow/simple_qa_agentic_flow.ipynb

# Install dependencies

In [1]:
%%capture --no-stderr
# %pip install -U --quiet 'crewai[tools]' aisuite

# Set environment variables

In [2]:
import os
OPENAI_API_KEY = open("/Users/mjack6/.secrets/openai_mjack.apikey", "r").read().strip()
os.environ["OPENAI_API_KEY"] = OPENAI_API_KEY

In [3]:
# Apply a patch to allow nested asyncio loops in Jupyter
import nest_asyncio
nest_asyncio.apply()

# Create Crew

In [4]:
# Importing Crew related components
# Importing CrewAI Flow related components
# Importing CrewAI Tools
from crewai import Agent, Task, Crew
from crewai.flow.flow import Flow, listen, start
from crewai_tools import WebsiteSearchTool

# Importing AI Suite for adhoc LLM calls and Pydantic
from pydantic import BaseModel
import aisuite as ai

/Users/mjack6/GSU_Spring2025/MSA8700/venv_agenticai/lib/python3.11/site-packages/pydantic/_internal/_config.py:295: PydanticDeprecatedSince20: Support for class-based `config` is deprecated, use ConfigDict instead. Deprecated in Pydantic V2.0 to be removed in V3.0. See Pydantic V2 Migration Guide at https://errors.pydantic.dev/2.10/migration/
  warnings.warn(DEPRECATION_MESSAGE, DeprecationWarning)
/Users/mjack6/GSU_Spring2025/MSA8700/venv_agenticai/lib/python3.11/site-packages/pydantic/_internal/_generate_schema.py:502: UserWarning: <built-in function callable> is not a Python type (it may be an instance of an object), Pydantic will allow any object with no validation since we cannot even enforce that the input is an instance of the given type. To get rid of this error wrap the type with `pydantic.SkipValidation`.
  warn(
/Users/mjack6/GSU_Spring2025/MSA8700/venv_agenticai/lib/python3.11/site-packages/crewai_tools/tools/scrapegraph_scrape_tool/scrapegraph_scrape_tool.py:34: PydanticDe

In [5]:
urls = [
    "https://lilianweng.github.io/posts/2023-06-23-agent/",
    "https://lilianweng.github.io/posts/2023-03-15-prompt-engineering/",
    "https://lilianweng.github.io/posts/2023-10-25-adv-attack-llm/",
]

research_agent = Agent(
    role="You are a helpful assistant that can answer questions about the web.",
    goal="Answer the user's question.",
    backstory="You have access to a vast knowledge base of information from the web.",
    tools=[
      WebsiteSearchTool(website=urls[0]),
      WebsiteSearchTool(website=urls[1]),
      WebsiteSearchTool(website=urls[2]),
    ],
    llm="gpt-4o-mini",
)

task = Task(
  description="Answer the following question: {question}",
  expected_output="A detailed and accurate answer to the user's question.",
  agent=research_agent,
)

crew = Crew(
    agents=[research_agent],
    tasks=[task],
)

Inserting batches in chromadb: 100%|██████████| 1/1 [00:01<00:00,  1.22s/it]


# Creating State

In [6]:
class QAState(BaseModel):
  """
  State for the documentation flow
  """
  question: str = "What does Lilian Weng say about the types of agent memory?"
  improved_question: str = ""
  answer: str = ""

# Creating Flow

In [7]:
class QAFlow(Flow[QAState]):
  @start()
  def rewrite_question(self):
    print(f"# Rewriting question: {self.state.question}")
    client = ai.Client()
    messages = [
        {
          "role": "system",
          "content": f"""Look at the input and try to reason about the underlying semantic intent / meaning.
            Here is the initial question:
            -------
            {self.state.question}
            -------
            Formulate an improved question:"""
        }
    ]

    response = client.chat.completions.create(
        model="openai:gpt-4o-mini",
        messages=messages,
        temperature=0.3
    )

    print(response)

    improved_question = response.choices[0].message.content
    self.state.improved_question = improved_question

  @listen(rewrite_question)
  def answer_question(self):
    print(f"# Answering question: {self.state.improved_question}")
    result = crew.kickoff(inputs={'question': self.state.improved_question})
    self.state.answer = result.raw
    return result


# Plotting Flow

In [10]:
flow = QAFlow()
flow.plot()

 
[2025-03-13 14:48:21][🌊 FLOW CREATED: 'QAFLOW']: 2025-03-13 14:48:21.181601
Plot saved as crewai_flow.html


In [15]:
# Display the flow visualization using HTML
from IPython.display import IFrame
IFrame(src='./crewai_flow.html', width='100%', height=400)

# Kicking off Flow

In [9]:
result = flow.kickoff()
print("=" * 10)
print(result)

 
[2025-03-13 14:47:33][🤖 FLOW STARTED: 'QAFLOW', D89FA34D-2656-4770-9164-E447D7CF8F25]: 2025-03-13 14:47:33.442886
 Flow started with ID: d89fa34d-2656-4770-9164-e447d7cf8f25
 
[2025-03-13 14:47:33][🤖 FLOW METHOD STARTED: 'REWRITE_QUESTION']: 2025-03-13 14:47:33.444760
# Rewriting question: What does Lilian Weng say about the types of agent memory?
ChatCompletion(id='chatcmpl-BAhsbtElvEOhcatNpI9Ee2Si96Pew', choices=[Choice(finish_reason='stop', index=0, logprobs=None, message=ChatCompletionMessage(content='What insights does Lilian Weng provide regarding the different types of agent memory?', refusal=None, role='assistant', audio=None, function_call=None, tool_calls=None, annotations=[]))], created=1741891653, model='gpt-4o-mini-2024-07-18', object='chat.completion', service_tier='default', system_fingerprint='fp_06737a9306', usage=CompletionUsage(completion_tokens=17, prompt_tokens=56, total_tokens=73, completion_tokens_details=CompletionTokensDetails(accepted_prediction_tokens=0, au